# Tratamento e geração do csv

In [ ]:
!pip install googlemaps
!pip install odfpy

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import googlemaps
from datetime import datetime
from google.colab import userdata

In [ ]:
# # Código de tratamento
# for ano in range(2015, 2020):
#   if f'Acidentes {ano}.ods' not in os.listdir():
#     !wget https://raw.githubusercontent.com/namespaiva/pi-acidentes/main/dados/Acidentes%20{ano}.ods

# for ano in range(2020, 2025):
#   if f'Acidentes {ano}.xls' not in os.listdir():
#     !wget https://raw.githubusercontent.com/namespaiva/pi-acidentes/main/dados/Acidentes%20{ano}.xls

# dados = pd.DataFrame()

# for ano in range (2015, 2020):
#   dados = pd.concat([dados, pd.read_excel(f'Acidentes {ano}.ods', engine='odf')])

# dados1 = dados.copy()

# for ano in range(2020, 2025):
#   dados = pd.concat([dados, pd.read_excel(f'Acidentes {ano}.xls')])

# df = dados.sort_values(by=['DATA', 'HORA'])

# df['CRUZAMENTO'] = df['CRUZAMENTO'].astype(str) + df['CRUZAMENTO.1'].astype(str)
# df['LOGRADOURO'] = df['LOGRADOURO'].astype(str) + df['LOGRADOURO1'].astype(str)
# df = df.drop(columns=['LOGRADOURO1', 'CRUZAMENTO.1', 'Unnamed: 0', 2])
# df.rename(columns={'Nº': 'NUMERO'}, inplace=True)
# df.to_csv('acidentes.csv', index=False)

# def remove_nans(s):
#     s = str(s)
#     if s == 'nan':
#         return s
#     elif s.startswith("nan"):
#         return s[3:]  # Skip the first 3 characters if they are "nan"
#     elif s.endswith("nan"):
#         return s[:-3]  # Skip the last 3 characters if they are "nan"
#     else:
#       return s

# df['LOGRADOURO'] = df['LOGRADOURO'].apply(remove_nans)
# df['CRUZAMENTO'] = df['CRUZAMENTO'].apply(remove_nans)

# df['DATA'] = pd.to_datetime(df['DATA'])
# df['NUMERO'] = df['NUMERO'].astype(str)
# df['lat'] = ''
# df['lng'] = ''
# df['types'] = ''
# df['bairro'] = ''
# df.rename(columns={'DATA': 'data', 'HORA': 'hora', 'TEMPO': 'tempo',
#                    'TIPO_ACIDENTE': 'tipo_acidente', 'GRAVIDADE': 'gravidade',
#                    'LOGRADOURO': 'logradouro', 'NUMERO': 'numero', 'CRUZAMENTO': 'cruzamento'}, inplace=True)

In [ ]:
# gravidades = list(df['gravidade'].unique())

# df['gravidade'] = df['gravidade'].apply(lambda x: 'nan' if pd.isna(x) else x)
# df['gravidade'] = df['gravidade'].apply(lambda x: 'nan' if x == ' ' else x)

In [ ]:
# gravidades = list(df['gravidade'].unique())
# gravidades.remove('nan')
# df['temp'] = df['cruzamento']

# new_gravidade_values = []
# new_temp_values = []

# for index, row in df.iterrows():
#     temp_value = row['temp']
#     for gravidade in gravidades:
#         if str(gravidade) in temp_value:
#             new_gravidade_values.append(str(gravidade))
#             temp_value = temp_value.replace(str(gravidade), '')
#             break
#     else:
#         new_gravidade_values.append(np.nan)

#     new_temp_values.append(temp_value)

# df['temp'] = new_temp_values

# df['cruzamento'] = df['temp']
# df.drop(columns=['temp'], axis=1, inplace=True)

In [ ]:
# df[df['data'].dt.year == 2021]
# df['cruzamento'] = df['cruzamento'].apply(lambda x: 'nan' if x == '' else x)
# df['cruzamento'].unique()

In [49]:
# !wget https://raw.githubusercontent.com/namespaiva/pi-acidentes/main/dados/Acidentes%202021.xls

# df = pd.read_csv('acidentes.csv')
# df['data'] = pd.to_datetime(df['data'])
# df.sort_values(by=['data', 'hora'], inplace=True)
# df.reset_index(drop=True, inplace=True)

# df2021 = pd.read_excel('Acidentes 2021.xls')

# mask_2021 = df['data'].dt.year == 2021

# # Check if the number of rows matches
# if len(df2021) == sum(mask_2021):
#     # Update 'gravidade' column where the year is 2021
#     df.loc[mask_2021, 'gravidade'] = df2021['CRUZAMENTO'].values
# else:
#     print("Erro filtrando df")


In [105]:
df = pd.read_csv('acidentes.csv')
df.sort_values(by=['data', 'hora'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [103]:
chave = userdata.get('chave')

gmaps = googlemaps.Client(key=chave)

In [104]:
for index, row in df.iterrows():
  if pd.isna(row['cruzamento']): # Use pd.isna() to check for NaN in any data type
    local = str(row['numero']) + ' ' + str(row['logradouro']) # Ensure concatenation compatibility
  else:
    local = str(row['logradouro']) + ' ' + str(row['cruzamento']) # Ensure concatenation compatibility

  geocode_result = gmaps.geocode(f'{local}, Santos, SP, Brazil')

  if pd.isna(row['cruzamento']):
    if len(geocode_result[0]['address_components']) > 2 and 'long_name' in geocode_result[0]['address_components'][2]:
      df.at[index, 'bairro'] = geocode_result[0]['address_components'][2]['long_name']
    else:
      print(index)
      df.at[index, 'bairro'] = 'Bairro não encontrado'
  else:
    if len(geocode_result[0]['address_components']) > 2 and 'long_name' in geocode_result[0]['address_components'][2]:
      df.at[index, 'bairro'] = geocode_result[0]['address_components'][1]['long_name']
    else:
      print(index)
      df.at[index, 'bairro'] = 'Bairro não encontrado'

  df.at[index, 'types'] = geocode_result[0]['types']
  df.at[index, 'lat'] = geocode_result[0]['geometry']['location']['lat']
  df.at[index, 'lng'] = geocode_result[0]['geometry']['location']['lng']

df.head(10)

ApiError: REQUEST_DENIED (The provided API key is expired. )

In [ ]:
# df.to_csv('acidentes.csv', index=False)

# Análise

In [ ]:
import folium
from folium.plugins import HeatMap
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

if 'acidentes.csv' not in os.listdir():
  !wget https://raw.githubusercontent.com/namespaiva/pi-acidentes/main/dados/acidentes.csv
  df = pd.read_csv('acidentes.csv')

df['data'] = pd.to_datetime(df['data'])
df.sort_values(by=['data', 'hora'], inplace=True)
df.reset_index(drop=True, inplace=True)

df.describe